## Accessing MoBI data with the Planetary Computer STAC API

The Map of Biodiversity Importance (MoBI) consists of a series of raster maps that combine habitat information for 2,216 imperiled species occurring in the conterminous United States, using weightings based on range size and degree of protection to identify areas of high importance for biodiversity conservation.

This notebook provides an example of accessing MoBI data from blob storage on Azure, using the Planetary Computer API.

Complete documentation for this dataset is available on the [Planetary Computer data catalog](https://planetarycomputer.microsoft.com/dataset/mobi).

### Environment setup

This notebook works with or without an API key, but you will be given more permissive access to the data with an API key. The Planetary Computer Hub is pre-configured to use your API key.

In [1]:
import pystac_client
import matplotlib.pyplot as plt
import numpy as np
import planetary_computer
import rasterio

### Data access

The datasets hosted by the Planetary Computer are available from [Azure Blob Storage](https://docs.microsoft.com/en-us/azure/storage/blobs/). We'll use [pystac-client](https://pystac-client.readthedocs.io/) to search the Planetary Computer's [STAC API](https://planetarycomputer.microsoft.com/api/stac/v1/docs) for the subset of the data that we care about, and then we'll load the data directly from Azure Blob Storage. We'll specify a `modifier` so that we can access the data stored in the Planetary Computer's private Blob Storage Containers. See [Reading from the STAC API](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/) and [Using tokens for data access](https://planetarycomputer.microsoft.com/docs/concepts/sas/) for more.

In [2]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

### Querying the dataset

Let's query the Planetary Computer to get all of the items within the `mobi` collection.

In [3]:
search = catalog.search(collections=["mobi"])

items = search.item_collection()

print(f"Returned {len(items)} Items")

Returned 1 Items


You'll see we only returned a single item for the entire collection. This is a bit different from other types of datasets on the Planetary Computer. In this case, the MoBI rasters are fairly low resolution, so tiling into separate rasters wasn't necessary. They also exist at a single temporal resolution, so each raster in the single item convers the entire CONUS.

Let's see what assets are associated with this item:

In [4]:
item = items[0]
print(*[f"{key}: {asset.description}" for key, asset in item.assets.items()], sep="\n")

RSR_All: Range-size rarity for species
RSR_Plants: Range-size rarity for vascular plants
RSR_Vertebrates: Range-size rarity for vertebrates
RSR_AquaticInverts: Range-size rarity for aquatic invertebrates
PWRSR_GAP12_SUM_All: Protection-weighted range-size rarity for all species
SpeciesRichness_All: Species richness for species
RSR_PollinatorInverts: Range-size rarity for pollinators
PWRSR_GAP12_SUM_Plants: Protection-weighted range-size rarity for vascular plants
SpeciesRichness_Plants: Species richness for vascular plants
PWRSR_GAP12_SUM_Vertebrates: Protection-weighted range-size rarity for vertebrates
SpeciesRichness_Vertebrates: Species richness for vertebrates
PWRSR_GAP12_SUM_AquaticInverts: Protection-weighted range-size rarity for aquatic invertebrates
SpeciesRichness_AquaticInverts: Species richness for aquatic invertebrates
PWRSR_GAP12_SUM_PollinatorInverts: Protection-weighted range-size rarity for pollinators
SpeciesRichness_PollinatorInverts: Species richness for pollinator

### Read and plot a layer

We've got 15 assets, which each correspond to a different raster. We'll pick the `SpeciesRichness_Vertebrates` asset and plot it by reading the entire file into memory directly from the blob store.

In [5]:
vertebreates_richness = item.assets["SpeciesRichness_Vertebrates"]

with rasterio.open(vertebreates_richness.href) as raster:
    data_array = raster.read(1).astype(float)
    raster.close()

# Set nodata values to be transparent
nd_val = raster.nodatavals[0]
data_array[data_array == nd_val] = np.nan

# Plot
fig = plt.figure(figsize=(12, 6), dpi=150, frameon=False)
plt.axis("off")
plt.imshow(data_array, cmap="terrain");

<Figure size 1800x900 with 1 Axes>